In [47]:
import requests
import pandas as pd
import numpy as np

### Parsing wikipedia table with pandas

In [146]:
df=pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")[0]
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [148]:
df.columns =["Postalcode","Borough","Neighborhood"] #correct the 1st column name
df.head()

,Postalcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


### Droping rows where Borough is not assigned

In [149]:
df = df[df.Borough != 'Not assigned'] 

### Merging Neighbourhoods by Postcode

In [152]:
for postcode in (df['Postalcode'].value_counts()>1).index.values:
    df.loc[df.Postalcode==postcode, 'Neighborhood'] = ', '.join(df[df.Postalcode==postcode]['Neighborhood'].values)
    
df.drop_duplicates(subset='Postalcode', keep="first", inplace = True)
df.head()

,Postalcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,"Lawrence Heights, Lawrence Manor"
7,M7A,Downtown Toronto,Queen's Park


### Assigning not assigned Neighbourhoods

In [154]:
df.loc[df['Neighborhood']=='Not assigned', 'Neighborhood'] = df[df['Neighborhood']=='Not assigned']['Borough']
df[df['Postalcode']=='M9A']

,Postalcode,Borough,Neighborhood
9,M9A,Queen's Park,Queen's Park


### Checking resulting dataframe shape

In [155]:
df.shape

(103, 3)

### Seting index

In [ ]:
df = df.set_index('Postalcode')

### Downloading geospatial data into a dataframe

In [185]:
geod = pd.read_csv('https://cocl.us/Geospatial_data')

geod.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Seting up geospatial columns on main dataframe

In [196]:

latitude = []
longitude = []

for i in df.index:
    latitude.append(geod[geod['Postal Code']==i]['Latitude'].values[0])
    longitude.append(geod[geod['Postal Code']==i]['Longitude'].values[0])

    
df['latitude'] = latitude
df['longitude'] = longitude


df.head()

,Borough,Neighborhood,latitude,longitude
Postalcode,,,,
M3A,North York,Parkwoods,43.753259,-79.329656
M4A,North York,Victoria Village,43.725882,-79.315572
M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
